In [3]:
import os
print(os.getcwd())
# change working directory to project root
import platform
if platform.system() is "Windows":
    os.chdir('D:\\git\\cardio')
else:
    os.chdir('/Users/minority/Code/Git/cardio')
print(os.getcwd())
from src.utils.utils_io import Console_and_file_logger
from src.utils.file_wrapper.app.Files import Basefile, Dicomfile
from src.utils.file_wrapper.app.IO import FileWalker
import logging
Console_and_file_logger('cleaning_all', logging.INFO)

import re
import nrrd
import numpy as np
from scipy import stats
from collections import OrderedDict
import json
import SimpleITK as sitk

/Users/minority/Code/Git/cardio/notebooks
/Users/minority/Code/Git/cardio


2018-11-10 00:35:00,409 INFO -------------------- Start --------------------
2018-11-10 00:35:00,410 INFO Filename: cleaning_all
2018-11-10 00:35:00,411 INFO Log directory: ./logs/


Working directory: /Users/minority/Code/Git/cardio.
Log dir: ./logs/


In [4]:
print(os.getcwd())
print('test')
experiment= 'tests'
filewalker = FileWalker(Dicomfile, src_dir='/Users/minority/Code/Git/cardio/data/testing/0006_filtered_flatten/', dest_dir='D:\\temp\\', describe=False)

logging.info('Filewalker done, writing json file...')   

with open(experiment + '.json', 'w') as outfile:
    json.dump({experiment : filewalker.open_files}, outfile, indent=2)
logging.info('json dump done, everything fine!') 

/Users/minority/Code/Git/cardio
test


2018-11-10 00:35:07,350 INFO Action performed in sub-directory /Users/minority/Code/Git/cardio/data/testing/0006_filtered_flatten/ done in 3.032s
2018-11-10 00:35:07,351 INFO Action performed on 433 files
2018-11-10 00:35:07,352 INFO Total Files: 433
2018-11-10 00:35:07,353 INFO Filewalker done, writing json file...
2018-11-10 00:35:07,358 INFO json dump done, everything fine!


In [3]:
# save the file walker as json
with open(experiment + '_filewalker.json', 'w') as outfile:
    json.dump({experiment + '_FileWalker' : filewalker.to_json()}, outfile, indent=2)
logging.info('filewalker dump done, everything fine!')    

2018-11-09 19:57:46,966 INFO filewalker dump done, everything fine!


In [9]:

# prove that len matched files = total files from file walker
sum([len(value) for key,value in filewalker.ignored_files.items()])

In [12]:
from collections import Counter
counter = Counter(filewalker.matched_files)

with open(experiment + '_counter_matched.json', 'w') as outfile:
    json.dump({experiment + '_counter_matched' : counter}, outfile, indent=2)
logging.info('counter dump done, everything fine!') 

2018-11-09 20:20:48,443 INFO counter dump done, everything fine!


In [23]:
class Dicomfile2(Dicomfile):


    def action(self):
        """
        Single entry point for the FileWalker
        Call here all functions you would like to perform
        :return:
        """
        logging.debug('action performed')

        return self.to_dcom_json()



    def to_dcom_json(self):
        """
        Returns a serialisable json representation
        :return:
        """
        
        sitk_img = self.sitk_img
        img = sitk.GetArrayFromImage(sitk_img)
        d = stats.describe(img.flatten())
        
        for k in sitk_img.GetMetaDataKeys():
            v = sitk_img.GetMetaData(k)
            #print("Key: {}, value: {}".format(k, str(v)))
            logging.info(k,str(v))

        
        def get_metadata_maybe(key):
            value = 'not_found'
            try:
                value = sitk_img.GetMetaData(key)
            except Exception as e:
                logging.debug('key not found: {}, {}'.format(key, e))
                pass
            
            return value
            
            
        
        
        def build_image():
            img = sitk.GetArrayFromImage(sitk_img)
            d = stats.describe(img.flatten())
            image = {}
            json_representation = {}
            json_representation['SOPInstanceUID'] = get_metadata_maybe('0008|0018')
            json_representation['img_min'] = int(d.minmax[0])
            json_representation['img_max'] = int(d.minmax[1])
            json_representation['img_mean'] = int(d.mean)
            json_representation['img_variance'] = int(d.variance)
            json_representation['sizes'] = str(sitk_img.GetSize())
            json_representation['dimension'] = int(sitk_img.GetDimension())
            json_representation['row'] = get_metadata_maybe('0028|0010')
            json_representation['column'] = get_metadata_maybe('0028|0011')
            json_representation['seriesinstanceuid'] = get_metadata_maybe('0020|000e')
            json_representation['SeriesDescription'] = get_metadata_maybe('0008|103e')                                                       
            json_representation['CardiacNumberOfImages'] = get_metadata_maybe('0018|1090')
            json_representation['MagneticFieldStrength'] = get_metadata_maybe('0018|0087')
            json_representation['SliceThickness'] = get_metadata_maybe('0018|0050')
            json_representation['PatientPosition'] = get_metadata_maybe('0018|5100')
            json_representation['SliceLocation'] = get_metadata_maybe('0020|1041')
            json_representation['SmallestImagePixelValue'] = get_metadata_maybe('0028|0106')
            json_representation['LargestImagePixelValue'] = get_metadata_maybe('0028|0107')
            json_representation['Manufacturer'] = get_metadata_maybe('0008|0070')
            json_representation['ManufacturerModelName'] = get_metadata_maybe('0008|1090')
            json_representation['InstitutionName'] = get_metadata_maybe('0008|0080')
            json_representation['InstitutionAddress'] = get_metadata_maybe('0008|0081')
            json_representation['ReferringPhysicianName'] = get_metadata_maybe('0008|0090')
            #json_representation['SOPInstanceUID'] = get_metadata_maybe('0008|0018')
            json_representation['num_images'] = int(1)
            json_representation['img_min'] = int(d.minmax[0])
            json_representation['img_max'] = int(d.minmax[1])
            json_representation['img_mean'] = int(d.mean)
            json_representation['img_variance'] = int(d.variance)
            json_representation['sizes'] = str(sitk_img.GetSize())
            json_representation['dimension'] = int(sitk_img.GetDimension())
            json_representation['row'] = get_metadata_maybe('0028|0010')
            json_representation['column'] = get_metadata_maybe('0028|0011')
            json_representation['studyinstanceuid'] = get_metadata_maybe('0020|000d')
            json_representation['StudyDescription'] = get_metadata_maybe('0008|1030')
            json_representation['PatientID'] = get_metadata_maybe('0010|0020')
            json_representation['PatientBirthDate'] = get_metadata_maybe('0010|0030')
            json_representation['PatientSex'] = get_metadata_maybe('0010|0040')
            json_representation['PatientAge'] = get_metadata_maybe('0010|1010')
            json_representation['PatientSize'] = get_metadata_maybe('0010|1020')
            json_representation['PatientWeight'] = get_metadata_maybe('0008|1140')
            #image[get_metadata_maybe('0008|0018')] = json_representation
            return json_representation
    
        def build_series():
            serie = {}
            json_representation = {}
            json_representation['seriesinstanceuid'] = get_metadata_maybe('0020|000e')
            json_representation['SeriesDescription'] = get_metadata_maybe('0008|103e')                                                       
            json_representation['CardiacNumberOfImages'] = get_metadata_maybe('0018|1090')
            json_representation['MagneticFieldStrength'] = get_metadata_maybe('0018|0087')
            json_representation['SliceThickness'] = get_metadata_maybe('0018|0050')
            json_representation['PatientPosition'] = get_metadata_maybe('0018|5100')
            json_representation['SliceLocation'] = get_metadata_maybe('0020|1041')
            json_representation['SmallestImagePixelValue'] = get_metadata_maybe('0028|0106')
            json_representation['LargestImagePixelValue'] = get_metadata_maybe('0028|0107')
            json_representation['Manufacturer'] = get_metadata_maybe('0008|0070')
            json_representation['ManufacturerModelName'] = get_metadata_maybe('0008|1090')
            json_representation['InstitutionName'] = get_metadata_maybe('0008|0080')
            json_representation['InstitutionAddress'] = get_metadata_maybe('0008|0081')
            json_representation['ReferringPhysicianName'] = get_metadata_maybe('0008|0090')
            #json_representation['SOPInstanceUID'] = get_metadata_maybe('0008|0018')
            json_representation['num_images'] = int(1)
            json_representation['img_min'] = int(d.minmax[0])
            json_representation['img_max'] = int(d.minmax[1])
            json_representation['img_mean'] = int(d.mean)
            json_representation['img_variance'] = int(d.variance)
            json_representation['sizes'] = str(sitk_img.GetSize())
            json_representation['dimension'] = int(sitk_img.GetDimension())
            json_representation['row'] = get_metadata_maybe('0028|0010')
            json_representation['column'] = get_metadata_maybe('0028|0011')
            #json_representation['Images'] = build_image()
            #serie[get_metadata_maybe('0020|000e')] = json_representation
            return [json_representation]
        
        def build_study():
            study = {}
            json_representation = {}
            json_representation['studyinstanceuid'] = get_metadata_maybe('0020|000d')
            json_representation['StudyDescription'] = get_metadata_maybe('0008|1030')
            json_representation['Series'] = build_series()
            #study[sitk_img.GetMetaData('0020|000d')] = json_representation
            return [json_representation]
        
        
        def build_patient():
            patient = {}
            json_representation = {}
            json_representation['PatientID'] = get_metadata_maybe('0010|0020')
            json_representation['PatientBirthDate'] = get_metadata_maybe('0010|0030')
            json_representation['PatientSex'] = get_metadata_maybe('0010|0040')
            json_representation['PatientAge'] = get_metadata_maybe('0010|1010')
            json_representation['PatientSize'] = get_metadata_maybe('0010|1020')
            json_representation['PatientWeight'] = get_metadata_maybe('0010|1030')
            json_representation['Studies'] = build_study()
            #patient['Patients'] = json_representation
            return {'Patients' : [json_representation]}
        
        
        return build_patient()
        #return build_image()

        

In [24]:
# test the Filewrapper

src = 'D:\\small\\dcm\\0001-02Z1L14N-2006-11-09\\1.3.12.2.1107.5.99.2.1013.30000008030612131778100062857\\'
filename = '1.3.12.2.1107.5.99.2.1013.30000008030612131778100062808.dcm'
dest = 'D:\\temp\\'
f_wrapper = Dicomfile2(src, filename, dest)
print(f_wrapper)
f_wrapper_json= f_wrapper.to_dcom_json()
print(json.dumps(f_wrapper_json, indent=4))

--- Logging error ---
Traceback (most recent call last):
  File "c:\users\1720267\anaconda3\Lib\logging\__init__.py", line 993, in emit
    msg = self.format(record)
  File "c:\users\1720267\anaconda3\Lib\logging\__init__.py", line 839, in format
    return fmt.format(record)
  File "c:\users\1720267\anaconda3\Lib\logging\__init__.py", line 576, in format
    record.message = record.getMessage()
  File "c:\users\1720267\anaconda3\Lib\logging\__init__.py", line 338, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "c:\users\1720267\anaconda3\Lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\1720267\anaconda3\Lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "d:\git\cardio\venv\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "d:\git\cardio\venv\lib\site-packages\traitlets\config\application.py", line 6

{
    "Patients": [
        {
            "PatientID": "02Z1L14N",
            "PatientBirthDate": "19000101",
            "PatientSex": "O ",
            "PatientAge": "106Y",
            "PatientSize": "1.65",
            "PatientWeight": "68",
            "Studies": [
                {
                    "studyinstanceuid": "1.3.12.2.1107.5.99.2.1013.30000008030612131778100067034",
                    "StudyDescription": "Herz_Routine^Stressperfusion",
                    "Series": [
                        {
                            "seriesinstanceuid": "1.3.12.2.1107.5.99.2.1013.30000008030612131778100062857",
                            "SeriesDescription": "axi6mm",
                            "CardiacNumberOfImages": "18",
                            "MagneticFieldStrength": "1.5 ",
                            "SliceThickness": "6 ",
                            "PatientPosition": "HFS ",
                            "SliceLocation": "68.646492004395 ",
                      

  File "d:\git\cardio\venv\lib\site-packages\IPython\core\interactiveshell.py", line 2809, in run_ast_nodes
    if self.run_code(code, result):
  File "d:\git\cardio\venv\lib\site-packages\IPython\core\interactiveshell.py", line 2869, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-24-6c9e69d580a8>", line 8, in <module>
    f_wrapper_json= f_wrapper.to_dcom_json()
  File "<ipython-input-23-ba50e47a79e2>", line 29, in to_dcom_json
    logging.info(k,str(v))
Message: '0032|1060'
Arguments: ('MR-Herz ',)
--- Logging error ---
Traceback (most recent call last):
  File "c:\users\1720267\anaconda3\Lib\logging\__init__.py", line 993, in emit
    msg = self.format(record)
  File "c:\users\1720267\anaconda3\Lib\logging\__init__.py", line 839, in format
    return fmt.format(record)
  File "c:\users\1720267\anaconda3\Lib\logging\__init__.py", line 576, in format
    record.message = record.getMessage()
  File "c:\users\1720267\anaconda3\Lib\logging\__init

In [13]:
a = {
}
b={
    "Patients": [
        {
            "PatientID": "02Z1L14N",
            "PatientBirthDate": "19000101",
            "PatientSex": "O ",
            "PatientAge": "106Y",
            "PatientSize": "1.65",
            "PatientWeight": "68",
            "Studies": [
                {
                    "studyinstanceuid": "1.3.12.2.1107.5.99.2.1013.30000008030612131778100067034",
                    "StudyDescription": "Herz_Routine^Stressperfusion",
                    "Series": [
                        {
                            "seriesinstanceuid": "1.3.12.2.1107.5.99.2.1013.30000008030612131778100062857",
                            "SeriesDescription": "axi6mm",
                            "CardiacNumberOfImages": "18",
                            "MagneticFieldStrength": "1.5 ",
                            "SliceThickness": "6 ",
                            "PatientPosition": "HFS ",
                            "SliceLocation": "68.646492004395 ",
                            "SmallestImagePixelValue": "8",
                            "LargestImagePixelValue": "2700",
                            "Manufacturer": "SIEMENS ",
                            "ManufacturerModelName": "Sonata",
                            "InstitutionName": "Radiologie Uniklinik T\udcfcbingen ",
                            "InstitutionAddress": "3Hoppe-Seyler-Strasse, Tuebingen/0d6498/, Stuttgart, 72076, Germany ",
                            "ReferringPhysicianName": "KI2AAMBU, Kinderkardiol. Ambulanz ",
                            "num_images": 1,
                            "img_min": 0,
                            "img_max": 2700,
                            "img_mean": 158,
                            "img_variance": 81458,
                            "sizes": "(192, 168, 1)",
                            "dimension": 3,
                            "row": "168",
                            "column": "192"
                        }
                    ]
                }
            ]
        }
    ]
}
c = {
    "Patients": [
        {
            "PatientID": "02Z1L14M",
            "PatientBirthDate": "19000101",
            "PatientSex": "O ",
            "PatientAge": "106Y",
            "PatientSize": "1.65",
            "PatientWeight": "68",
            "Studies": [
                {
                    "studyinstanceuid": "1.3.12.2.1107.5.99.2.1013.30000008030612131778100067034",
                    "StudyDescription": "Herz_Routine^Stressperfusion",
                    "Series": [
                        {
                            "seriesinstanceuid": "1.3.12.2.1107.5.99.2.1013.30000008030612131778100062857",
                            "SeriesDescription": "axi6mm",
                            "CardiacNumberOfImages": "18",
                            "MagneticFieldStrength": "1.5 ",
                            "SliceThickness": "6 ",
                            "PatientPosition": "HFS ",
                            "SliceLocation": "68.646492004395 ",
                            "SmallestImagePixelValue": "8",
                            "LargestImagePixelValue": "2700",
                            "Manufacturer": "SIEMENS ",
                            "ManufacturerModelName": "Sonata",
                            "InstitutionName": "Radiologie Uniklinik T\udcfcbingen ",
                            "InstitutionAddress": "3Hoppe-Seyler-Strasse, Tuebingen/0d6498/, Stuttgart, 72076, Germany ",
                            "ReferringPhysicianName": "KI2AAMBU, Kinderkardiol. Ambulanz ",
                            "num_images": 1,
                            "img_min": 0,
                            "img_max": 2700,
                            "img_mean": 158,
                            "img_variance": 81458,
                            "sizes": "(192, 168, 1)",
                            "dimension": 3,
                            "row": "168",
                            "column": "192"
                        }
                    ]
                }
            ]
        }
    ]
}
d = {
    "Patients": [
        {
            "PatientID": "02Z1L14M",
            "PatientBirthDate": "19000101",
            "PatientSex": "O ",
            "PatientAge": "106Y",
            "PatientSize": "1.65",
            "PatientWeight": "68",
            "Studies": [
                {
                    "studyinstanceuid": "1.3.12.2.1107.5.99.2.1013.30000008030612131778100067030",
                    "StudyDescription": "Herz_Routine^Stressperfusion",
                    "Series": [
                        {
                            "seriesinstanceuid": "1.3.12.2.1107.5.99.2.1013.30000008030612131778100062857",
                            "SeriesDescription": "axi6mm",
                            "CardiacNumberOfImages": "18",
                            "MagneticFieldStrength": "1.5 ",
                            "SliceThickness": "6 ",
                            "PatientPosition": "HFS ",
                            "SliceLocation": "68.646492004395 ",
                            "SmallestImagePixelValue": "8",
                            "LargestImagePixelValue": "2700",
                            "Manufacturer": "SIEMENS ",
                            "ManufacturerModelName": "Sonata",
                            "InstitutionName": "Radiologie Uniklinik T\udcfcbingen ",
                            "InstitutionAddress": "3Hoppe-Seyler-Strasse, Tuebingen/0d6498/, Stuttgart, 72076, Germany ",
                            "ReferringPhysicianName": "KI2AAMBU, Kinderkardiol. Ambulanz ",
                            "num_images": 1,
                            "img_min": 0,
                            "img_max": 2700,
                            "img_mean": 158,
                            "img_variance": 81458,
                            "sizes": "(192, 168, 1)",
                            "dimension": 3,
                            "row": "168",
                            "column": "192"
                        }
                    ]
                }
            ]
        }
    ]
}

In [ ]:

        
def describe(a, indent=""):
    if isinstance(a, dict):
        for key, value in a.items():
            print(indent + key)
            if isinstance(value, list):
                describe(value[0], indent + "   ")

                
        
describe(b)

In [14]:
def merge(a, b, path=None):
    "merges b into a"
    if path is None: path = []
    for key in b:
        if key in a:
            # if value is a dict, recursive
            if isinstance(a[key], dict) and isinstance(b[key], dict):
                merge(a[key], b[key], path + [str(key)])

            elif key == "num_images":
                a[key] = a[key] + 1

            # values are similar, keep it
            elif a[key] == b[key]:
                pass
            else:
                # values from different images, sumarize them     
                if key == "img_min":
                    a[key] = min(a[key], b[key])

                elif key == "img_max":
                    a[key] = max(a[key], b[key])

                elif key == "img_mean":
                    a[key] = int((a[key] + b[key]) / 2)

                elif key == "img_variance":
                    a[key] = int((a[key] + b[key]) / 2)

                elif key == "SmallestImagePixelValue":
                    a[key] = min(a[key], b[key])

                elif key == "LargestImagePixelValue":
                    a[key] = max(a[key], b[key])
                else:
                    # overwrite value in a for value in b (this will be the fact to sumarize images in a serie)
                    a[key] = b[key]
                    #raise Exception('Conflict at %s' % '.'.join(path + [str(key)]))
           
        # key from b is not in a
        else:
            a[key] = b[key]
    return a
a_b = merge(a,b)
a_b_c = merge(a_b, c)
a_b_c_d = merge(a_b_c, d)
print(json.dumps(a_b_c_d, indent=2))

{
  "Patients": [
    {
      "PatientID": "02Z1L14M",
      "PatientBirthDate": "19000101",
      "PatientSex": "O ",
      "PatientAge": "106Y",
      "PatientSize": "1.65",
      "PatientWeight": "68",
      "Studies": [
        {
          "studyinstanceuid": "1.3.12.2.1107.5.99.2.1013.30000008030612131778100067030",
          "StudyDescription": "Herz_Routine^Stressperfusion",
          "Series": [
            {
              "seriesinstanceuid": "1.3.12.2.1107.5.99.2.1013.30000008030612131778100062857",
              "SeriesDescription": "axi6mm",
              "CardiacNumberOfImages": "18",
              "MagneticFieldStrength": "1.5 ",
              "SliceThickness": "6 ",
              "PatientPosition": "HFS ",
              "SliceLocation": "68.646492004395 ",
              "SmallestImagePixelValue": "8",
              "LargestImagePixelValue": "2700",
              "Manufacturer": "SIEMENS ",
              "ManufacturerModelName": "Sonata",
              "InstitutionName": 

In [4]:
# create single dicomfiles to test the merge method
a_src = 'D:\\small\\dcm\\0001-02Z1L14N-2006-11-09\\1.3.12.2.1107.5.99.2.1013.30000008030612131778100062857\\'
a_filename = '1.3.12.2.1107.5.99.2.1013.30000008030612131778100062808.dcm'
b_src = 'D:\\small\\dcm\\0001-02Z1L14N-2006-11-09\\1.3.12.2.1107.5.99.2.1013.30000008030612131778100062857\\'
b_filename = '1.3.12.2.1107.5.99.2.1013.30000008030612131778100062805.dcm'
c_src = 'D:\\small\\dcm\\0002-02Z1L14N-2006-11-09\\1.3.12.2.1107.5.2.12.21128.30000006110907310151500003217\\'
c_filename = '1.3.12.2.1107.5.2.12.21128.30000006110907310151500003218.dcm'
d_src = 'D:\\small\\dcm\\0120-5R8HZG7H-2007-01-08\\1.3.46.670589.11.0.0.11.4.2.0.5167.5.2868.2007010812585362609\\'
d_filename = '1.3.46.670589.11.0.0.11.4.2.0.5167.5.2868.2007010813013484678.dcm'
dest = 'D:\\temp\\'

a = Dicomfile(a_src, a_filename, dest).to_dcom_json()
b = Dicomfile(b_src, b_filename, dest).to_dcom_json()
c = Dicomfile(c_src, c_filename, dest).to_dcom_json()
d = Dicomfile(d_src, d_filename, dest).to_dcom_json()
a

{'Patients': [{'PatientAge': '106Y',
   'PatientBirthDate': '19000101',
   'PatientID': '02Z1L14N',
   'PatientSex': 'O ',
   'PatientSize': '1.65',
   'PatientWeight': '68',
   'Studies': [{'Series': [{'CardiacNumberOfImages': '18',
       'InstitutionAddress': '3Hoppe-Seyler-Strasse, Tuebingen/0d6498/, Stuttgart, 72076, Germany ',
       'InstitutionName': 'Radiologie Uniklinik T\udcfcbingen ',
       'LargestImagePixelValue': 2714,
       'MagneticFieldStrength': '1.5 ',
       'Manufacturer': 'SIEMENS ',
       'ManufacturerModelName': 'Sonata',
       'PatientPosition': 'HFS ',
       'ReferringPhysicianName': 'KI2AAMBU, Kinderkardiol. Ambulanz ',
       'SeriesDescription': 'axi6mm',
       'SliceLocation': '68.646492004395 ',
       'SliceThickness': '6 ',
       'SmallestImagePixelValue': 9,
       'column': '192',
       'dimension': 3,
       'img_max': 2714,
       'img_mean': 160,
       'img_min': 0,
       'img_variance': 81323,
       'num_images': 1,
       'row': '168'

In [4]:
# merging nested json files
def merge2(a, b):
    "merges b into a"
    if len(a.keys()) is 0:
        a = b
    
    merge_p_id = b['Patients'][0]['PatientID']
    merge_st_id = b['Patients'][0]['Studies'][0]['studyinstanceuid']
    merge_se_id = b['Patients'][0]['Studies'][0]['Series'][0]['seriesinstanceuid']
    new_patient=True
    new_study=True
    new_serie=True
    
    #print(merge_p_id, merge_st_id, merge_se_id)
    
    # check if patient is knows
    for idx_p,patient in enumerate(a['Patients']):
        if merge_p_id == patient['PatientID']:
            new_patient = False
            break # stop after first match
    if new_patient:
        a['Patients'].append(b['Patients'][0])
    else: # patient is known, check if study is new
        for idx_st,study in enumerate(a['Patients'][idx_p]['Studies']):
            if merge_st_id == study['studyinstanceuid']:
                new_study = False
                break
        if new_study:
            a['Patients'][idx_p]['Studies'].append(b['Patients'][0]['Studies'][0])
        else: # patient and study are known, check if serie is new
            for idx_se,serie in enumerate(a['Patients'][idx_p]['Studies'][idx_st]['Series']):
                if merge_se_id == serie['seriesinstanceuid']:
                    new_serie = False
                    break
            if new_serie:
                a['Patients'][idx_p]['Studies'][idx_st]['Series'].append(b['Patients'][0]['Studies'][0]['Series'][0])
            else: # patient, study and serie are known, sumarize this image to the known image-data
                a_serie = a['Patients'][idx_p]['Studies'][idx_st]['Series'][idx_se]
                b_serie = b['Patients'][0]['Studies'][0]['Series'][0]

                a_serie['num_images'] +=1
                a_serie['img_min'] = min(a_serie['img_min'], b_serie['img_min'])
                a_serie['img_min'] = max(a_serie['img_max'], b_serie['img_max'])
                a_serie['img_min'] = int((a_serie['img_mean'] + b_serie['img_mean']) / 2)
                a_serie['img_min'] = int((a_serie['img_variance'] + b_serie['img_variance']) / 2)
                a_serie['img_min'] = min(a_serie['SmallestImagePixelValue'], b_serie['SmallestImagePixelValue'])
                a_serie['img_min'] = max(a_serie['LargestImagePixelValue'], b_serie['LargestImagePixelValue'])
            
    return a
    

merged = merge2(a,b)
#print(json.dumps(merged, indent=2))
merged2 = merge2(merged, c)
#print(json.dumps(merged2, indent=2))
merged3 = merge2(merged2, d)
print(json.dumps(merged3, indent=2))
#print(json.dumps(merge(a,c), indent=2))

{
  "Patients": [
    {
      "PatientID": "02Z1L14N",
      "PatientBirthDate": "19000101",
      "PatientSex": "O ",
      "PatientAge": "106Y",
      "PatientSize": "1.65",
      "PatientWeight": "68",
      "Studies": [
        {
          "studyinstanceuid": "1.3.12.2.1107.5.99.2.1013.30000008030612131778100067034",
          "StudyDescription": "Herz_Routine^Stressperfusion",
          "Series": [
            {
              "seriesinstanceuid": "1.3.12.2.1107.5.99.2.1013.30000008030612131778100062857",
              "SeriesDescription": "axi6mm",
              "CardiacNumberOfImages": "18",
              "MagneticFieldStrength": "1.5 ",
              "SliceThickness": "6 ",
              "PatientPosition": "HFS ",
              "SliceLocation": "68.646492004395 ",
              "SmallestImagePixelValue": 9,
              "LargestImagePixelValue": 2714,
              "Manufacturer": "SIEMENS ",
              "ManufacturerModelName": "Sonata",
              "InstitutionName": "Rad

In [20]:
import json
json1_file = open('dcm_all_lists.json')
json1_str = json1_file.read()
json1_data = json.loads(json1_str)

In [19]:
def depth(d, level=1):
    if not isinstance(d, dict) or not d:
        return level
    return max(depth(d[k], level + 1) for k in d)

In [21]:

key_value = [(key, value) for key,value in json1_data.items()]
print(len(key_value))
print(depth(json1_data))
#print(json.dumps(key_value[1], indent=1))
keys = [key for key,value in json1_data.items()]
print(keys)
data_new = {'ahf_all': key_value}
with open('ahf_all_new.json', 'w') as outfile:
    json.dump(data_new, outfile, indent=1)


1


NameError: name 'depth' is not defined